# **05. Generalize Model**

**Topic**: Generalize your Model by adjusting the validation method, preprocessing, feature engineering, and so on.

In [1]:
import numpy as np

data = np.random.rand(5000)

### <font color='dark'>**Cut our Dataset into usages for training, testing and validation**</font>

<p>模型的架構(多少層,多深), 這些參數稱為 <font color='blue'><b>Hyperparameter</b></font> (超參數)</p>
<p>那像是神經網路的權重參數, 這些參數稱為 <font color='blue'><b>Weight parameter</b></font> (權重參數)</p>

<p>我們都知道 Weight parameter 會隨我們訓練來調整, 那 Hyperparameter 怎麼辦呢?</p>
<p>這時就是我們 validation data 的功用了, 我們可以看 validation 的結果來調整我們的模型架構</p>
<p>但這樣調整也怕過多, 造成模型去擬合我們的 validation data, 這種現象稱為 <font color='blue'><b>information leak</b></font>(資訊洩漏)</p>
<p>當然我們也不想用 test data 來調整我們的 hyperparameter, 讓我們的 model 去學習 test data 就失去意義了</p>
-------------------------------------------------------------

這邊提供幾種 validation data 驗證方法
<ul>
    <li><font color='dark'>Simple hold-out validation</font></li>
    <li><font color='dark'>K-fold validation</font></li>
    <li><font color='dark'>Iterated K-fold validation with shuffling</font></li>
</ul>

#### **Simple hold-out validation**

這種方法適合在你有很多資料的時候來使用, 概念很簡單,

我們就乖乖的切 3 種 data, 分別是 train, test, validation, 

而 val data 通常是從 train 切出去的, 切記 val 不能太少, 不然不具有統計意義

In [2]:
num_val_data = 1000

np.random.shuffle(data)

val_data = data[:num_val_data]
train_data = data[num_val_data:]

經過 val data 調整完我們的參數後, 會再把 val 和 train 做合併再一起訓練

In [3]:
data = np.concatenate([train_data, val_data])

#### **K-fold validation**

<img src='https://scikit-learn.org/stable/_images/grid_search_cross_validation.png'>

這方法適用於資料少的情況

將我們的 train data 切成 K 折, 並以 K-1 折作為我們的 train data, 並以其中 1 折來作為我們的 val data,

而這會重複 K 次來進行, 那每次選的 val data 會輪流來當, 最後我們把這些 K 次的 validation scores 來取平均作為我們的參考依據

In [4]:
k = 4
num_val_data = len(data)//k

np.random.shuffle(data)

val_scores = []
for fold in range(k):
    val_data = data[ num_val_data*fold : num_val_data*(fold+1) ]
    train_data = np.concatenate([data[ :num_val_data*fold ], data[ num_val_data*(fold+1): ] ])
    
    # 用 model evaluate 來算出 val scores 加到我們的 list
    score = 88
    val_scores.append(score)

#### **Iterated K-fold validation with shuffling**

此方法適用於資料相對較少, 且需要盡可能精確的評估模型的情況

這是目前在 Kaggle 常使用的方法, 簡單來講就是進行很多次的 K fold, 而每次資料都必須進行 shuffle, 

假設進行 P 次, 那麼就是說進行 P*K 回的訓練, 這方法相對來說運算成本相當的高

#### **Validation 注意事項**

* **資料代表性**
    * 假設我們要判斷 0～9 的資料, 我們不能把 label 0~7 為訓練, 8～9 為val, 這樣訓練就不具代表性, 因此我們需要 **randomly shuffle** 來處理資料
* **時間方向性**
    * 當處理時間序的資料, 我們界不能隨機取了, 因為這樣會造成 temporal leak (時間漏失) 發生時間錯位的狀況, 因此我們要確保 test data 都在 train data 之後
* **資料重複現象**
    * 有時候同樣的 label 及 feature 會出現在 train 及 val data 兩次, 這樣會造成表現不可信的狀況產生, 我們要避免